<a href="https://colab.research.google.com/github/BrianSward/colab-work/blob/main/extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook go fetch your article, extract the text content of the website, format it, and return this information to the user.

This will bypass attempts to block you the user from accessing news that is of value to you.

It also removes ALL NON TEXT, for good or bad.

In [ ]:
!pip install spacy
!pip install trafilatura

In [39]:
# imports for the web scraping

from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura
import textwrap

In [43]:
user_url = input("Provide me a URL: ")

text_data = {}
resp = requests.get(user_url)
if resp.status_code == 200:
  text_data[user_url] = resp.text

def beautifulsoup_extract_text_fallback(response_content):
        
    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')
    
    # Finding the text:
    text = soup.find_all(text=True)
    
    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)
            
    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()
    

def extract_text_from_single_web_page(url):
    
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments = False,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, json_output=True, with_metadata=True,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan
          
single_url = user_url
_text = extract_text_from_single_web_page(url=single_url)


# split the text into paragraphs
paragraphs = str(_text).split("\n")

# wrap each paragraph to a maximum of 80 characters per line
for paragraph in paragraphs:
    wrapped_paragraph = textwrap.wrap(paragraph, width=80)
    # print the wrapped paragraph lines
    for line in wrapped_paragraph:
        print(line)
    # print a blank line to separate paragraphs
    print()

Provide me a URL: https://www.washingtonpost.com/national-security/2023/04/22/justice-department-fines-pardons-gupta/
The Justice Department sent a memo to the nation’s judges on Thursday that
warned against imposing high fees and fines that would unfairly burden low-
income people.

Both efforts were part of the Biden administration’s campaign to address what
Justice Department officials describe as systemic failings in the justice system
that disproportionately affect the poor and people of color.

“Imposing and enforcing fines and fees on individuals who cannot afford to pay
them has been shown to cause profound harm,” officials wrote in the “Letter to
Colleagues” memo this week, which warned that harsh financial penalties that
trap people in poverty could be unconstitutional.

“Individuals confront escalating debt; face repeated, unnecessary incarceration
for nonpayment of fines and fees; experience extended periods of probation and
parole; are subjected to changes in immigration s